In [6]:
import json
import pandas as pd
import pickle

In [7]:
def cve_matches_name(cve, name):
    for vendor in cve['cve']['affects']['vendor']['vendor_data']:
        for product in vendor['product']['product_data']:
            if product['product_name'] == name:
                return True
    return False



In [23]:
def find_vulns(product_name): #version_str):
    results = []
    for year in range(10,19):
        j = json.load(open('/Users/ashton/school/cmsc828m/project/data/nvd_json/nvdcve-1.0-20' + str(year) + '.json'))
        for cve in j["CVE_Items"]:
            #if j["CVE_Items"][i]["cve"]["CVE_data_meta"]['ID'] in cve_ids:
            #    results.append(j["CVE_Items"][i])
            if cve_matches_name(cve, product_name):
                results.append(cve)
        del j
    print("unfiltered list: ", product_name, len(results))
    return results

def find_versions(all_vulns, version_str):
    results2 = []
    for r in all_vulns:
        versions = []
        for vd in r['cve']['affects']['vendor']['vendor_data']:
            for pd in vd['product']['product_data']:
                for version in pd['version']['version_data']:
                    #print version
                    versions.append(version['version_value'])
        if version_str in versions:
            results2.append({"is" : r['impact']['baseMetricV2']['impactScore'], 
                             "es" : r["impact"]['baseMetricV2']["exploitabilityScore"],
                             "bs" : r["impact"]['baseMetricV2']["cvssV2"]["baseScore"],
                             "cve_id" : r["cve"]["CVE_data_meta"]["ID"]})
    #results[0]['cve']['affects']['vendor']['vendor_data']#['version_data']
    print("final filtered: ", version_str, len(results2))
    return results2

#django_cves = find_vulns("php")
#len(django_cves)
#django_1_7_cves = find_versions(django_cves, "1.7")

In [16]:
def get_attacks(attacker_skill_level, attack_set):
    return [a for a in attack_set if a["es"] <= attacker_skill_level]

def get_attacks_rel(percentile, attack_set):
    return sorted(attack_set, key=lambda x: x['es'])[:int(len(attack_set) * percentile)]

#get_attacks(7, django_1_7_cves)

In [20]:
product_cves = {}
for product in ['jenkins', 'mongodb', 'php', 'django', 'postgresql',
               "mysql", "tomcat" , "http_server", "drupal", "moodle", "iis", "wordpress", "squid", 
               "memcached", "hadoop", "jira"]:
    product_cves[product] = find_vulns(product)    

('unfiltered list: ', 'jenkins', 94)
('unfiltered list: ', 'mongodb', 12)
('unfiltered list: ', 'php', 318)
('unfiltered list: ', 'django', 57)
('unfiltered list: ', 'postgresql', 55)
('unfiltered list: ', 'mysql', 504)
('unfiltered list: ', 'tomcat', 91)
('unfiltered list: ', 'httpd', 0)
('unfiltered list: ', 'drupal', 87)
('unfiltered list: ', 'moodle', 292)
('unfiltered list: ', 'iis', 10)
('unfiltered list: ', 'wordpress', 162)
('unfiltered list: ', 'squid', 40)
('unfiltered list: ', 'memcached', 11)
('unfiltered list: ', 'hadoop', 17)
('unfiltered list: ', 'jira', 19)


In [42]:
product_cves['http_server'] = find_vulns('http_server')

('unfiltered list: ', 'http_server', 68)


In [43]:
versions = {'django' : "1.7", 
            "mongodb" :  "2.4.3",
            "php" : "7.0.0" ,
            "postgresql" : "9.5",
            "jenkins" : "1.625.1",
           "mysql" : "5.5.19",
           "tomcat" : "6.0.10",
           "http_server" : "2.0.39",
           "drupal" : "6.0", 
           "moodle" : "2.1",
           "iis" : "6.0",
           "wordpress" : "2.0",
           "squid" : "3.2.0.3",
           "memcached" : "1.4.15",
           "hadoop" : "2.0.0",
           "jira" : "3.12"}


In [44]:
product_version_cves = {}
for k, v in versions.items():
    product_version_cves[k] = find_versions(product_cves[k], v)

#product_version_cves['php']

('final filtered: ', '3.12', 9)
('final filtered: ', '9.5', 16)
('final filtered: ', '6.0', 3)
('final filtered: ', '6.0.10', 46)
('final filtered: ', '6.0', 38)
('final filtered: ', '2.4.3', 4)
('final filtered: ', '3.2.0.3', 25)
('final filtered: ', '2.0.0', 6)
('final filtered: ', '2.1', 110)
('final filtered: ', '1.7', 19)
('final filtered: ', '2.0', 37)
('final filtered: ', '2.0.39', 15)
('final filtered: ', '5.5.19', 159)
('final filtered: ', '1.625.1', 16)
('final filtered: ', '7.0.0', 85)
('final filtered: ', '1.4.15', 4)


In [46]:
# get rewards based on relative rankings (top 50%, top 75% etc.)
rewards = []
attacker_skill = [0.5, 0.75, 1]
services = sorted(versions.keys())
for s in services:
    theta = []
    for a in attacker_skill:
        theta.append(get_attacks_rel(a, product_version_cves[s]))
    rewards.append(theta)
    

In [62]:
# get rewards based on absolute skill (ES score thresholds)
rewards = []
attacker_skill = [6, 8, 10]
services = sorted(versions.keys())
for s in versions.keys():
    theta = []
    for a in attacker_skill:
        theta.append(get_attacks(a, product_version_cves[s]))
    rewards.append(theta)
    

In [63]:
# removing attacks which are strictly dominated 
new_rewards = []
for s in range(len(rewards)):
    new_rewards.append([])
    for theta in range(len(rewards[s])):
        new_rewards[s].append([])
        already_added = []
        for a_1 in rewards[s][theta]:
            if not any(a_2['bs'] >= a_1['bs'] and a_2['es'] < a_1['es'] for a_2 in rewards[s][theta]) and \
                    not (a_1['bs'], a_1['es']) in already_added:
                new_rewards[s][theta].append(a_1)
                already_added.append((a_1['bs'], a_1['es']))

In [64]:
results = []
for i, s in enumerate(services):
    d = {"service" : s, "version" : versions[s], "a1" : len(new_rewards[i][0]), "a2" : len(new_rewards[i][1]), "a3" : len(new_rewards[i][2]),\
        "original vulns" : len(product_version_cves[s])}
    d['undominated vulns'] = d['a1'] + d['a2'] + d['a3']
    results.append(d)
    
df = pd.DataFrame(results, columns=["service", 'version', 'original vulns', 'undominated vulns', 'a1', 'a2', 'a3'])
df

,service,version,original vulns,undominated vulns,a1,a2,a3
0,django,1.7,19,4,0,2,2
1,drupal,6.0,38,8,2,3,3
2,hadoop,2.0.0,6,3,0,1,2
3,http_server,2.0.39,15,14,3,4,7
4,iis,6.0,3,13,3,5,5
5,jenkins,1.625.1,16,5,1,2,2
6,jira,3.12,9,7,1,1,5
7,memcached,1.4.15,4,7,1,2,4
8,mongodb,2.4.3,4,15,2,6,7
9,moodle,2.1,110,8,1,2,5


In [65]:
pickle.dump(rewards, open('/Users/ashton/school/cmsc828m/project/data/attacker_actions/20180429-rewards.pickle', 'wb'))

In [66]:
pickle.dump(services,  open('/Users/ashton/school/cmsc828m/project/data/attacker_actions/20180429-services.pickle', 'wb'))

In [68]:
df.to_csv('/Users/ashton/school/cmsc828m/project/data/attacker_actions/20180429-rewards-table.csv', index=False)

In [69]:
with open('/Users/ashton/school/cmsc828m/project/data/attacker_actions/20180429-rewards-table.csv') as f:
    s = f.read()
s

'service,version,original vulns,undominated vulns,a1,a2,a3\ndjango,1.7,19,4,0,2,2\ndrupal,6.0,38,8,2,3,3\nhadoop,2.0.0,6,3,0,1,2\nhttp_server,2.0.39,15,14,3,4,7\niis,6.0,3,13,3,5,5\njenkins,1.625.1,16,5,1,2,2\njira,3.12,9,7,1,1,5\nmemcached,1.4.15,4,7,1,2,4\nmongodb,2.4.3,4,15,2,6,7\nmoodle,2.1,110,8,1,2,5\nmysql,5.5.19,159,14,1,5,8\nphp,7.0.0,85,12,3,3,6\npostgresql,9.5,16,24,5,9,10\nsquid,3.2.0.3,25,3,1,1,1\ntomcat,6.0.10,46,4,0,0,4\nwordpress,2.0,37,5,1,2,2\n'

In [70]:
print(s.replace(',', " & ").replace("\n", "\\tabularnewline\n"))

service & version & original vulns & undominated vulns & a1 & a2 & a3\tabularnewline
django & 1.7 & 19 & 4 & 0 & 2 & 2\tabularnewline
drupal & 6.0 & 38 & 8 & 2 & 3 & 3\tabularnewline
hadoop & 2.0.0 & 6 & 3 & 0 & 1 & 2\tabularnewline
http_server & 2.0.39 & 15 & 14 & 3 & 4 & 7\tabularnewline
iis & 6.0 & 3 & 13 & 3 & 5 & 5\tabularnewline
jenkins & 1.625.1 & 16 & 5 & 1 & 2 & 2\tabularnewline
jira & 3.12 & 9 & 7 & 1 & 1 & 5\tabularnewline
memcached & 1.4.15 & 4 & 7 & 1 & 2 & 4\tabularnewline
mongodb & 2.4.3 & 4 & 15 & 2 & 6 & 7\tabularnewline
moodle & 2.1 & 110 & 8 & 1 & 2 & 5\tabularnewline
mysql & 5.5.19 & 159 & 14 & 1 & 5 & 8\tabularnewline
php & 7.0.0 & 85 & 12 & 3 & 3 & 6\tabularnewline
postgresql & 9.5 & 16 & 24 & 5 & 9 & 10\tabularnewline
squid & 3.2.0.3 & 25 & 3 & 1 & 1 & 1\tabularnewline
tomcat & 6.0.10 & 46 & 4 & 0 & 0 & 4\tabularnewline
wordpress & 2.0 & 37 & 5 & 1 & 2 & 2\tabularnewline

